In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold

import os
import random
import warnings
warnings.filterwarnings("ignore")

In [2]:
np.random.seed(2022)
random.seed(2022)
os.environ['PYTHONHASHSEED'] = '0'

In [3]:
data = pd.read_csv('./creditcard.csv')

In [4]:
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [5]:
data.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,...,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.859575,3.918649e-15,5.682686e-16,-8.761736e-15,2.811118e-15,-1.552103e-15,2.040130e-15,-1.698953e-15,-1.893285e-16,-3.147640e-15,...,1.473120e-16,8.042109e-16,5.282512e-16,4.456271e-15,1.426896e-15,1.701640e-15,-3.662252e-16,-1.217809e-16,88.349619,0.001727
std,47488.145955,1.958696e+00,1.651309e+00,1.516255e+00,1.415869e+00,1.380247e+00,1.332271e+00,1.237094e+00,1.194353e+00,1.098632e+00,...,7.345240e-01,7.257016e-01,6.244603e-01,6.056471e-01,5.212781e-01,4.822270e-01,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,...,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.500000,-9.203734e-01,-5.985499e-01,-8.903648e-01,-8.486401e-01,-6.915971e-01,-7.682956e-01,-5.540759e-01,-2.086297e-01,-6.430976e-01,...,-2.283949e-01,-5.423504e-01,-1.618463e-01,-3.545861e-01,-3.171451e-01,-3.269839e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84692.000000,1.810880e-02,6.548556e-02,1.798463e-01,-1.984653e-02,-5.433583e-02,-2.741871e-01,4.010308e-02,2.235804e-02,-5.142873e-02,...,-2.945017e-02,6.781943e-03,-1.119293e-02,4.097606e-02,1.659350e-02,-5.213911e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.500000,1.315642e+00,8.037239e-01,1.027196e+00,7.433413e-01,6.119264e-01,3.985649e-01,5.704361e-01,3.273459e-01,5.971390e-01,...,1.863772e-01,5.285536e-01,1.476421e-01,4.395266e-01,3.507156e-01,2.409522e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,9.382558e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,...,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519589e+00,3.517346e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


In [6]:
data.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [7]:
print('Class 0:', data['Class'].value_counts()[0]) # No Frauds
print('Class 1:', data['Class'].value_counts()[1]) # Frauds

Class 0: 284315
Class 1: 492


In [9]:
print("No frauds rate : {}".format((284315 / (284315 + 492)) * 100))

No frauds rate : 99.82725143693798


In [10]:
# Scaling
from sklearn.preprocessing import RobustScaler

rob_scaler = RobustScaler()

data['amount'] = rob_scaler.fit_transform(data['Amount'].values.reshape(-1,1))
data['time'] = rob_scaler.fit_transform(data['Time'].values.reshape(-1,1))

data.drop(['Time', 'Amount'], axis=1, inplace=True)

In [11]:
data.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V22,V23,V24,V25,V26,V27,V28,Class,amount,time
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,...,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,0,1.783274,-0.994983
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,...,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,0,-0.269825,-0.994983
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,...,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,0,4.983721,-0.994972
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,...,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,0,1.418291,-0.994972
4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,...,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,0,0.670579,-0.994960


In [12]:
# Undersampling

In [14]:
data1 = data.sample(frac=1) # data random shuffle

fraud_data = data1.loc[data1['Class'] == 1]
non_fraud_data = data1.loc[data1['Class'] == 0][:492]

In [15]:
print(len(fraud_data))
print(len(non_fraud_data))

492
492


In [16]:
balanced_data = pd.concat([fraud_data, non_fraud_data])
balanced_data = balanced_data.sample(frac=1)

In [17]:
balanced_data.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V22,V23,V24,V25,V26,V27,V28,Class,amount,time
213005,1.787155,-1.617071,-1.140874,-0.937252,-0.791878,0.097439,-0.749371,0.025273,-0.215018,0.822725,...,0.550282,-0.024122,0.307207,-0.203156,-0.223969,-0.046904,-0.023155,0,2.640956,0.638906
10484,1.088375,0.898474,0.394684,3.170258,0.175739,-0.221981,-0.022989,-0.010874,0.860044,-0.592473,...,-0.800852,0.077614,0.167608,0.350182,-0.118941,0.012948,0.054254,1,-0.254454,-0.793066
215678,2.148323,0.540882,-3.527269,0.564761,1.453125,-1.212508,0.672031,-0.311852,0.348500,-0.999488,...,-0.249410,-0.100089,-0.195055,0.417648,0.733449,-0.091336,-0.024408,0,-0.296793,0.651770
243547,-6.618211,3.835943,-6.316453,1.844111,-2.476892,-1.886718,-3.817495,0.613470,-1.482121,-4.868747,...,0.038727,0.278218,0.786670,0.063895,0.154707,-2.042403,1.405141,1,0.499266,0.790423
154693,-2.488363,4.359019,-7.776410,5.364027,-1.823877,-2.445140,-4.964221,1.484890,-2.947899,-7.175350,...,1.021226,-0.266476,-0.370880,0.365535,0.081372,0.184983,-0.211582,1,-0.293440,0.210611


In [18]:
X = balanced_data.drop('Class', axis=1)
y = balanced_data['Class']

In [19]:
# Data Split
from sklearn.model_selection import train_test_split

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

X_train = X_train.values
X_test = X_test.values
y_train = y_train.values
y_test = y_test.values

In [21]:
logistic = LogisticRegression()
logistic.fit(X_train, y_train)

LogisticRegression()

In [22]:
from sklearn.model_selection import cross_val_score

training_score = cross_val_score(logistic, X_train, y_train, cv=5)

In [23]:
training_score

array([0.94936709, 0.93670886, 0.92993631, 0.93630573, 0.91082803])

In [25]:
# 평가
y_pred = logistic.predict(X_test)

from sklearn.metrics import accuracy_score
test_accuracy = accuracy_score(y_test, y_pred)
test_accuracy

0.949238578680203

In [26]:
# Oversampling - SMOTE

In [27]:
X = data.drop('Class', axis=1)
y = data['Class']

In [28]:
X.shape

(284807, 30)

In [29]:
kfold = StratifiedKFold(n_splits=5)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

X_train = X_train.values
X_test = X_test.values
y_train = y_train.values
y_test = y_test.values

In [30]:
print(X_train.shape)
print(X_test.shape)

(227845, 30)
(56962, 30)


In [31]:
from imblearn.over_sampling import SMOTE

In [32]:
logistic = LogisticRegression()

In [34]:
for train, test in kfold.split(X_train, y_train):
    smote = SMOTE(sampling_strategy='minority')
    x_oversample, y_oversample = smote.fit_resample(X_train[train], y_train[train])
    
    logistic.fit(x_oversample, y_oversample)

In [35]:
y_pred = logistic.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred)
print(test_accuracy)

0.9752817667918964
